# Information Theory Lab 03: Recognize Text Language with Memory Sources

## About

This file is designed to be viewed and run online in a browser.

This file is a Jupyter Notebook file usign `xeus-cling`, a Jupyter kernel for C++ based on the `cling` C++ interpreter and the `xeus` native implementation of the Jupyter protocol, xeus.

- GitHub repository: https://github.com/jupyter-xeus/xeus-cling/
- Online documentation: https://xeus-cling.readthedocs.io/ 

<!-- <img src="images/xeus-cling.png" alt="xeus-cling logo" style="width: 100px;"/> -->

## Usage

To run a selected code cell:

- Ctrl  + Enter = Run cell and remain at current cell
- Shift + Enter = Run cell and advance to next cell

<!--
<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>
-->
<br>

# 1. Objective

Understand the concept of source with memory and use it in a nice application.

# 2. Practical considerations

## 2.1 Minor prerequisites

Some simple function we need are presented below.

The function `isalpha()` returns non-zero if a character is a letter (a to z or A to Z), and 0 if not. Run the cell below to play with it.

In [ ]:
isalpha('b') != 0

The function `tolower()` converts an upper-case letter into a lower-case one. A non-letter is left unchanged.

In [ ]:
tolower(';') == ';'

`strlen()` can be used to determine the length of a string of characters.

In [ ]:
strlen("Salut")

## 2.2 Building a source with memory

We consider a source with memory, which can produces all 26 letters of the English alphabet (`a` to `z`, only lower-case). 
The source has memory order = 1, i.e. it remembers the previous letter. 
The total number of states is 26.

The transition matrix $T$ is a $26 \times 26$ matrix which holds the transition probability from one state to another, i.e. from one letter to the next one:
 
- $T[0,0]$ = probability that an `a` is followed by a `a`
- $T[0,1]$ = probability that an `a` is followed by a `b`
- ...
- $T[i][j]$ = probability that the $i$-th letter of the alphabet is followed by the $j$-th letter of the alphabet
- ...
- $T[25,25]$ = probability that a `z` is followed by a `z`

Different languages have different transition probabilities.

How to find these transition probabilities? We can estimate them by counting letters from a large text file. as follows. 
We have a $26 \times 26$ matrix $C$ of counters. Every time we see an `a` followed by an `a`, we increment $C[0][0]$. 
When we see an `a` followed by a `b`, we increment $C[0][1]$, and so on. Essentially we count every group of two consecutive letters.

Once we have the counted values, we find probabilities by dividing to the total.

### Step 1: count pairs of letters

Consider the following text. Go through the `text` array with a `for` loop, and increment the counter matrix `count` appropriately for every pair of consecutive letters.
Notes:
  - upper-case letters are not considered, we convert them to lower-case with `tolower()`
  - we ignore non-letters. Use `isalpha()` to check that both characters are actually letters
  - we initialize all the counters **with 1 and not 0**, to avoid having probability any transition with zero probability

In [ ]:
int count[26][26];

// Initialize all counters with 1
for (int i=0; i<26; i++)
    for (int j=0; j<26; j++)
        count[i][j] = 1;

const char* text = "Congratulations, you won the lottery!";

// TODO: write below




// Display the count matrix obtained
count

### Step 2: compute transition matrix

Compute the transition matrix $T$ by dividing each row of `count` to the sum of that row. Every row in the matrix $T$ should sum up to 1.

In [ ]:
double T[26][26]; // will hold the transition probabilities

// TODO: write below




// Display here the resulting matrix
T

### Exercise 3: Putting it all together into a function

Write a function `void getT(const char* fn, double T[][26])` to compute the transition matrix from a large text file.
- the function receives the name of the file as the first argument `fn`
- it opens the file, reads every byte, increments counters (only for letters)
- it computes the transition probabilities and stores them in the matrix `T` received as second argument

In [ ]:
void getT(const char* fn, double T[][26])
{
    // Some necessary variables here
    FILE* f;
    unsigned char c1, c2;
    int count[26][26];
    int total=0;
    
    // TODO: write below

    
    
}

Now let's call the function on the files `textro.txt` and `texten.txt` and see the results

In [ ]:
double T_ro[26][26];        // for Romanian language
double T_en[26][26];        // for English language
getT("textro.txt", T_ro);   // Romanian text file
getT("texten.txt", T_en);   // English text file

// display one of the matrices
T_ro

### Step 4: Compute probability of an unknown text

Given a text array `text` in an unknown language, we detect the language by calculating and comparing the probability that it comes from the Romanian language model vs the the probability that it comes from the English language model.

We compute the probability of the text by multiplying the transition probabilities for every pair of letters in the sequence. We obtain the total probability of the sequence, for each language model, and then we compare them.

We take a decision based on the higher probability. This is known as **Maximum Likelihood** principle: we pick the alternative which is more likely to have produced the text, i.e. the language for which the text probability is higher.

Multiplying probabilities rapidly produces values too small to be represented in binary format. To avoid this, instead of multiplying transition some probabilities 
$$P = p_1 \cdot p_2 \cdot ... $$
we instead sum their logarithms $$S  = log10(p1) + log10(p2) + ...$$

This keeps the values in an acceptable range. In the end we compare the two total sums just like we would have compared the total probabilities.

Compute below the probabilities (log sums) `prob_ro` and `prob_en` for the given text below, according to the Romanian and English transition matrices `T_ro` and `T_en`.

In [2]:
const char* text = "Where are you, Scooby-Dooby-Doo?";
//const char* text = "Unde esti tu, Azorel?";

double prob_ro = 0;  // sums must be initialized with 0
double prob_en = 0;

// TODO: write below, compute prob_ro and prob_en



### Results

Finally, let's compare the total log-probabilities and guess the language!

In [ ]:
printf("Prob RO = %g, Prob EN = %g\n", prob_ro, prob_en);

if (prob_ro > prob_en)
    printf("Looks like Romanian text");
else
    printf("Looks like English text");


# 3. Final Exercises

1. Put it all into a dedicated C program.

    Write a C program *guesstext.c* guess the language of a text file.

    * The program shall receive the names of three files:
    
        `guesstext.exe textro.txt texten.txt unknown.txt
    
        The three arguments are:
    
         * a large text file written in Romanian language
         
         * a large text file written in English language
         
         * a text file written either in Romanian or in English
                    
    * The program should follow the same steps as above:
    
      - create the transition matrices for the two languages, with the function created above
      
      - open the unknown file, read the text, compute the probabilities, and finally report the language in the console



# 4. Final questions

1. TBD